In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,6431
2,Huelva,Confirmados PDIA 14 días,2422
3,Huelva,Tasa PDIA 14 días,"474,2111002989762"
4,Huelva,Confirmados PDIA 7 días,1073
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,13
629,Municipio de Huelva sin especificar,Total Confirmados,107
630,Municipio de Huelva sin especificar,Curados,21


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  6431.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  1758.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 425 personas en los últimos 7 días 

Un positivo PCR cada 212 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,6431.0,1073.0,2422.0,510743.0,210.086090,474.211100,214.0
Huelva-Costa,3328.0,634.0,1293.0,288115.0,220.051021,448.779133,133.0
Condado-Campiña,2020.0,277.0,723.0,155057.0,178.643983,466.280142,74.0
Huelva (capital),1758.0,338.0,678.0,143663.0,235.272826,471.937799,74.0
Sierra de Huelva-Andévalo Central,977.0,149.0,361.0,67571.0,220.508798,534.252860,25.0
Aljaraque,249.0,44.0,93.0,21260.0,206.961430,437.441204,18.0
Palma del Condado (La),296.0,42.0,110.0,10761.0,390.298299,1022.209832,14.0
Gibraleón,285.0,54.0,146.0,12607.0,428.333466,1158.086777,14.0
Valverde del Camino,201.0,30.0,93.0,12820.0,234.009360,725.429017,12.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Aracena,246.0,73.0,146.0,8107.0,900.456396,1800.912791,9.0
Escacena del Campo,85.0,7.0,39.0,2284.0,306.479860,1707.530648,1.0
Gibraleón,285.0,54.0,146.0,12607.0,428.333466,1158.086777,14.0
Palma del Condado (La),296.0,42.0,110.0,10761.0,390.298299,1022.209832,14.0
Villablanca,34.0,20.0,24.0,2848.0,702.247191,842.696629,5.0
Lepe,504.0,94.0,208.0,27431.0,342.677992,758.266195,12.0
Niebla,68.0,10.0,31.0,4117.0,242.895312,752.975468,6.0
Valverde del Camino,201.0,30.0,93.0,12820.0,234.009360,725.429017,12.0
Villarrasa,70.0,4.0,15.0,2176.0,183.823529,689.338235,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cortegana,123.0,1.0,10.0,4666.0,21.431633,214.316331,0.0,0.100000
Calañas,34.0,1.0,10.0,2791.0,35.829452,358.294518,0.0,0.100000
Alosno,35.0,3.0,21.0,3923.0,76.472088,535.304614,0.0,0.142857
Escacena del Campo,85.0,7.0,39.0,2284.0,306.479860,1707.530648,1.0,0.179487
Almonaster la Real,30.0,2.0,9.0,1819.0,109.950522,494.777350,0.0,0.222222
San Silvestre de Guzmán,5.0,1.0,4.0,623.0,160.513644,642.054575,0.0,0.250000
Villarrasa,70.0,4.0,15.0,2176.0,183.823529,689.338235,1.0,0.266667
Trigueros,102.0,11.0,38.0,7713.0,142.616362,492.674705,4.0,0.289474
Almonte,307.0,27.0,93.0,24191.0,111.611756,384.440494,3.0,0.290323
